# Algoritmo de búsqueda para selección de características

* **Javier Ruiz Muñiz**
* **Jesús Elías Rodríguez**

En este archivo se realizarán las implementaciones de los algoritmos SFS(Sequential forward selection) y SFFS(Sequential floating forward selection), siendo probadas y contrastando los resultados obtenidos para los dos conjuntos de datos proporcionados en la asignatura de IA.

Para la correcta implementación de los algoritmos citados, tuvimos que realizar un algoritmo de **validación robusta**, haciendo uso del framework de Python llamado Scikit-Learn, que nos proporcionó la función **cross_val_score** que nos permitió realizar validaciones mediante validación cruzada.

La implementación sería la siguiente:

In [4]:
def validacionRobusta(X,y,CV,hitRate):
    
    n_exp = 10  
  
    scoreI = list()
    accuracies = dict()

    for i in range(n_exp):
        tree = DecisionTreeClassifier()
        
        #Aplicamos validación cruzada
        scores = cross_val_score(tree,X,y,cv=CV,scoring=hitRate)
        #print(scores)
        avgScores = np.mean(scores)
        accuracies.update({i:avgScores})

    avgFinal = sum(accuracies.values())/n_exp
    return avgFinal

# SFS(Sequential forward selection)

En primer lugar, debemos sacar los elementos del fichero **CSV** e introducirlos en un **DataFrame** para poder trabajar con ellos de manera más eficiente. Además introduciremos las importaciones utilizadas para la realización de los algoritmos:

In [5]:
import pandas as pd
import numpy as np
import math
import sklearn
import operator

import matplotlib.pyplot as plt
from sklearn import tree, linear_model
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from collections import OrderedDict


##Sacamos los datos del fichero csv
datos = pd.read_csv('titanic.csv', sep=',', index_col=0)
#datos = pd.read_csv('BreastCancerDataset.csv', sep=',', index_col=0)
dataFrame = pd.DataFrame(datos)

Una vez obtenidos los datos, podemos empezar a trabajar con ellos.

La implementación del algoritmo SFS es la siguiente:

In [8]:
#Devolver tabla con variables por k y su respectivo rendimiento
def sfs(answerVar, predictorVar, D):
    CV = 3
    hitRate = 'balanced_accuracy'
    
    solucionActual = pd.DataFrame()
    
    solucion = dict()
    
    supportDataFrame = predictorVar
   
    if D is None:
        D = len(predictorVar.columns)
     
    k=1
    while(k<=D):
    
        
        lastValue = 0
        solucionTemporal = pd.DataFrame();
        
        for j in range(0, len(supportDataFrame.columns)):
            X = pd.concat([solucionActual,supportDataFrame.iloc[:,j]], axis=1)
            y = answerVar
            if k==1:
                validation = validacionRobusta(X.to_numpy().reshape(-1,1),y,CV,hitRate)
            else:
                validation = validacionRobusta(X,y,CV,hitRate)
                
            if(validation>lastValue):
                lastValue = validation
                solucionTemporal = X
        
        #Eliminamos del supportDataFrame la ultima variable añadida a la solución
        lastColumn = solucionTemporal.columns[k-1]
        del supportDataFrame[lastColumn]
        
        
        #Actualizamos la solución actual
        solucionActual = solucionTemporal
        solucion.update({tuple(solucionActual.columns):validation})
       
        k = k+1
        
     #Creamos un DataFrame para mostrar los resultados
    solucion=OrderedDict(sorted(solucion.items(),key=operator.itemgetter(1), reverse=True))
        
    df = pd.DataFrame([[key, solucion[key], len(key)] for key in solucion.keys()], columns=['Solution', 'Score','Size'])
        
    return df

Procederemos a continuación a realizar los test correspondientes:


In [ ]:
answerVar = dataFrame.iloc[:,-1]
predictorVar = dataFrame.iloc[:,0:-1]
D=None

sfs(answerVar, predictorVar, D)

# SFFS(Sequential floating forward selection)

Al igual del apartado anterior, debemos sacar los elementos del fichero **CSV** e introducirlos en un **DataFrame** para poder trabajar con ellos de manera más eficiente. Además introduciremos las importaciones utilizadas para la realización de los algoritmos:

In [11]:
import pandas as pd
import numpy as np
import operator
from collections import OrderedDict


from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier


##Sacamos los datos del fichero csv
datos = pd.read_csv('titanic.csv', sep=',', index_col=0)
#datos = pd.read_csv('BreastCancerDataset.csv', sep=',', index_col=0)

dataFrame = pd.DataFrame(datos)

A continuación, procedemos a realizar el algoritmo SFFS:

In [12]:
#Creación del algoritmo SFFS
def sffs(answerVar, predictorVar):
    
    CV = 3
    hitRate = 'balanced_accuracy'
    
    solucionActual = pd.DataFrame()
    supportDataFrame = predictorVar
    
    añadidos = list()
    eliminados = list()
    solucion = dict()
    solucionFinal=dict()
    
    
    k=0
    
    while(k<len(predictorVar.columns)):
        
        mejorSolucionTemporal = seleccionaMejorVariable(supportDataFrame, solucionActual,k,CV,hitRate)
          
        solucionActual = mejorSolucionTemporal[0]
        rendimiento = mejorSolucionTemporal[1]
        añadidos.append(mejorSolucionTemporal[2])
        
        #Si procesamos más de una variable por iteración, miramos si podemos eliminar alguna para mejorar el rendimiento
        if k!=0:
            res = eliminaSiHayMejora(solucionActual, rendimiento, CV, hitRate, eliminados)
            solucionActual= res[0]
            eliminados= res[1]
            validation= res[2]
            #Guardamos todas las soluciones en un diccionario
            solucion.update({tuple(solucionActual.columns):validation})
        
          #Eliminamos del supportDataFrame la mejor variable
        for c in range (0, len(añadidos)):
            nombreColumna = añadidos[c]
            if nombreColumna in supportDataFrame.columns:
                supportDataFrame=supportDataFrame.drop(nombreColumna, axis=1)
               
        
        
        
        k=k+1
    #Creamos un DataFrame para mostrar los resultados de cada iteración
    solucionSinParada=mostrarSolucion(solucion)
    
     #Comparamos elementos de añadidos con las variables predictoras para poder lanzar la condición de parada
    añadidosList= sorted(añadidos)
    predictorVarList= sorted(predictorVar.columns)
    
    if añadidosList == predictorVarList:
        res=condicionDeParada(eliminados,solucionActual,validation,CV, hitRate)
        solucionActual=res[0]
        rendimiento=res[1]
    
    #Creamos un DataFrame para mostrar la mejor solución después de la condición de parada.
    solucionFinal.update({tuple(solucionActual.columns):rendimiento})
    solucionConParada=mostrarSolucion(solucionFinal)
   
    return (solucionSinParada, solucionConParada)

def seleccionaMejorVariable(supportDataFrame, solucionActual, k,CV,hitRate):
    
    lastValue = 0
    bestSolucionTemporal = pd.DataFrame()
    bestColumn = ''
    
    for j in range(0, len(supportDataFrame.columns)):
            
            solucionTemporal = supportDataFrame.iloc[:,j]
            X = pd.concat([solucionActual,solucionTemporal], axis=1)
            y = answerVar
            
            if k==0:
                validation = validacionRobusta(X.to_numpy().reshape(-1,1),y,CV,hitRate)
            else:
                validation = validacionRobusta(X,y,CV,hitRate)
                
            if(validation>lastValue):
                lastValue = validation
                bestSolucionTemporal = X
                bestColumn = solucionTemporal.name
                
    
    return (bestSolucionTemporal, lastValue, bestColumn)
    
    
def eliminaSiHayMejora(solucionActual, rendimiento, CV, hitRate, eliminados):
    lastValue2 = 0
    
    for j in range(0, len(solucionActual.columns)):
        
        solucionTemporal=solucionActual
        solucionTemporal=solucionTemporal.drop([solucionTemporal.columns[j]], axis=1)
        X = solucionTemporal
        y = answerVar
        
        if len(solucionActual.columns) == 2:
            validation = validacionRobusta(X.to_numpy().reshape(-1,1),y,CV,hitRate)
        else:
            validation = validacionRobusta(X,y,CV,hitRate)
        
        if(validation>lastValue2):
            lastValue2 = validation
            mejorSolucionTemporal = X
            colunmDeleted = solucionActual.iloc[:,j]
            eliminadoTemporal = colunmDeleted.name
        
    if lastValue2>rendimiento:
        validation=lastValue2
        solucionActual = mejorSolucionTemporal     
        eliminados.append(eliminadoTemporal)
    else:
        validation=rendimiento
    
    return (solucionActual, eliminados, validation)

def mostrarSolucion(solucion):
    solucionConParada = pd.DataFrame([[key, solucion[key], len(key)] for key in solucion.keys()], columns=['Solution', 'Score','Size'])
    solucionConParada = solucionConParada.sort_values('Score',ascending=False)
    
    return solucionConParada
 
def condicionDeParada(eliminados,solucionActual,validation,CV, hitRate):
    c=0
    eliminadosSinParada=eliminados[:]
    while(c<10):
        res = eliminaSiHayMejora(solucionActual, validation, CV, hitRate, eliminados) 
        solucionActual = res[0]
        rendimiento = res[2]
        if eliminadosSinParada == eliminados:
            c=c+1
            
        else:
            c=0
            eliminadosSinParada = eliminados[:]
    return (solucionActual,rendimiento )



Procedemos a realizar los test correspondientes. En el primero mostraremos la solución sin aplicar la condición de parada, y en el segundo, si se aplicaría:

In [13]:
answerVar = dataFrame.iloc[:,-1]
predictorVar = dataFrame.iloc[:,0:-1]
solucion = sffs(answerVar, predictorVar)
solucionSinParada = solucion[0]
solucionSinParada

,Solution,Score,Size
4,"(SibSp, Deck, Title, Is_Married, Sex)",0.816197,5
2,"(Initial, SibSp, Deck, Title)",0.813551,4
3,"(Initial, SibSp, Deck, Title, Is_Married)",0.813369,5
1,"(Initial, SibSp, Deck)",0.811260,3
0,"(Initial, SibSp)",0.805388,2


In [16]:
solucionConParada = solucion[1]
if solucionSinParada.iloc[0].all() == solucionConParada.iloc[0].all():
        print('No se ha conseguido mejorar el resultado con la condicion de parada')
else:
     print('Si se ha conseguido mejorar el resultado con la condicion de parada')

solucionConParada

No se ha conseguido mejorar el resultado con la condicion de parada


,Solution,Score,Size
0,"(SibSp, Deck, Title, Is_Married, Sex)",0.816197,5
